In [1]:
from muutils.zanj import ZANJ
from maze_transformer.generation.generators import LatticeMazeGenerators
from maze_transformer.training.maze_dataset import MazeDataset, MazeDatasetConfig, MAZE_DATASET_CONFIGS

print(MAZE_DATASET_CONFIGS.keys())

dict_keys(['test-g3-n5-a_dfs'])


In [2]:
zanj: ZANJ = ZANJ(external_list_threshold=16)
cfg: MazeDatasetConfig = MazeDatasetConfig(
	name="test",
	grid_n=3,
	n_mazes=32,
	maze_ctor=LatticeMazeGenerators.gen_dfs,
)

In [3]:
dataset: MazeDataset = MazeDataset.from_config(
	cfg,
	do_download=False,
	load_local=False,
	local_base_path="../data/",
	verbose=True,
	zanj=zanj,
)

generating & solving mazes: 100%|██████████| 32/32 [00:07<00:00,  4.08maze/s]


In [4]:
dataset_cpy: MazeDataset = MazeDataset.from_config(
	cfg,
	do_download=False,
	load_local=True,
	do_generate=False,
	local_base_path="../data/",
	verbose=True,
	zanj=zanj,
)


In [5]:
assert dataset.cfg == dataset_cpy.cfg
print(len(dataset.mazes), len(dataset_cpy.mazes))
print(dataset_cpy.mazes[0])
assert all(x == y for x,y in zip(dataset.mazes, dataset_cpy.mazes))

32 32
SolvedMaze(connection_list=array([[[ True,  True,  True],
        [ True, False,  True],
        [False, False, False]],

       [[False, False, False],
        [False,  True, False],
        [ True,  True, False]]]), generation_meta=None, lattice_dim=2, solution=array([[2, 0],
       [2, 1],
       [2, 2]]))
